In [99]:
#General library modules
import pandas as pd
import os
import glob
from pathlib import Path
import email
import email.policy
import email.parser
from bs4 import BeautifulSoup as bs
import nltk
import re
import string
import chardet

In [100]:
ham_files = [name for name in sorted(os.listdir('./ham'))]
spam_files = [name for name in sorted(os.listdir('./spam'))]

In [101]:
len(ham_files), len(spam_files)

(2551, 500)

In [102]:
spam_files[10], ham_files[100]

('0011.2a1247254a535bac29c476b86c708901',
 '0101.48557f7f38d947eab77aefc03d0520a3')

### Downloading Mails 

In [103]:
def extract_mails(path, mails):
    with open(os.path.join(path, mails), 'rb') as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [104]:
ham_emails = [extract_mails('./ham', ham) for ham in ham_files]

In [105]:
ham_emails[2002].get_content()

"URL: http://boingboing.net/#85485775\nDate: Not supplied\n\nNewZoid generates fake news-headlines: \n\n    1- Miss Universe Calling For Isidore \n    2- Rumsfeld Suggests Aspirin Reduces Alzheimer's Risk \n    3- Editorial Attacks Computer System \n    4- Stressed Out? Just Call For New TV Show \n    5- Gore To Hear About It \n    6- Nurses To Be Given Their Own Passports On Iraq Action \n    7- US Arraigned In Fighting Slavery \n    8- Lisa Riley Lines Below Chaos Above  \n\nLink[1] Discuss[2] (_Thanks, Daniel[3]!_)\n\n[1] http://www.newzoid.com/nzheadlinecombo.asp\n[2] http://www.quicktopic.com/boing/H/8juRS7A4p6JY\n[3] http://www.newzoid.com\n\n\n"

In [106]:
spam_email = [extract_mails('./spam', spam) for spam in spam_files]

In [107]:
spam_email[10].get_content()

'Cellular Phone Accessories All At Below Wholesale Prices!\n\nhttp://202.101.163.34:81/sites/merchant/sales/\n\nHands Free Ear Buds 1.99! \nPhone Holsters 1.98! \nBooster Antennas Only $0.99\nPhone Cases 1.98! \nCar Chargers 1.98! \nFace Plates As Low As 0.99! \nLithium Ion Batteries As Low As 6.94! \n\nhttp://202.101.163.34:81/sites/merchant/sales/\n\nClick Below For Accessories On All NOKIA, MOTOROLA LG, NEXTEL, \nSAMSUNG, QUALCOMM, ERICSSON, AUDIOVOX PHONES At Below \nWHOLESALE PRICES!\n\nhttp://202.101.163.34:81/sites/merchant/sales/\n\n***If You Need Assistance Please Call Us (732) 751-1457***\n\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\nTo be removed from future mailings please send your remove \nrequest to: removemenow68994@btamail.net.cn \nThank You and have a super day :)\n\n'

In [108]:
def cleanHtml(soup):
    for s in soup(['script', 'style']):
        s.decompose()
    cleanSoup = ' '.join(soup.stripped_strings)
    return cleanSoup

In [109]:
def extract_msg(emails):

    #lists to store email messages
    msgText = []
    text = ''
    #for mail in emails:
    if emails.is_multipart():
        html = None
        #for part in emails.walk():
        for part in emails.get_payload():    
            if part.get_content_charset() is None:
                charset = chardet.detect(str(part).encode())#['encoding']
            else:
                charset = part.get_content_charset()
                
            if part.get_content_type() == 'text/plain':
                text = str(part.get_payload(decode=True),str('iso-8859-1'),"ignore").encode('utf-8','replace')
                
            if part.get_content_type() == 'text/html':
                msg = str(part.get_payload(decode=True),str('iso-8859-1'),"ignore").encode('utf-8','replace')
                soup = bs(msg, 'lxml')
                html = cleanHtml(soup)

            if part.get_content_type() == 'multipart/alternative':
                for subpart in part.get_payload():
                    if subpart.get_content_charset() is None:
                        charset = chardet.detect(str(subpart))['encoding']
                    else:
                        charset = subpart.get_content_charset()
                        
                    if subpart.get_content_type() == 'text/html':
                        msg = str(subpart.get_payload(decode=True),str(charset),"ignore").encode('utf-8','replace')
                        soup = bs(msg, 'lxml')
                        html = cleanHtml(soup)
                        
                    if subpart.get_content_type() == 'text/plain':
                        text = str(subpart.get_payload(decode=True),str(charset),"ignore").encode('utf-8','replace')
            
            if html is None:
                text.strip()
                msgText.append(text)
            else:
                html.strip()
                msgText.append(html)
    else:
        text = str(emails.get_payload(decode=True), 'iso-8859-1','ignore').encode('utf-8','replace')
        text.strip()
        msgText.append(text)
            
    return msgText   

In [110]:
ham_mail = [extract_msg(x) for x in ham_emails]

In [111]:
spam_mail = [extract_msg(x) for x in spam_email]

In [112]:
len(ham_mail), len(spam_mail)

(2551, 500)

In [113]:
hamdf = pd.DataFrame(ham_mail).T

In [114]:
hamdf = hamdf.T
hamdf

,0,1,2
0,"b' Date: Wed, 21 Aug 2002 10:54:46 -...",None,None
1,"b""Martin A posted:\nTassos Papadopoulos, the G...",None,None
2,"b""Man Threatens Explosion In Moscow \n\nThursd...",None,None
3,"b""Klez: The Virus That Won't Die\n \nAlready t...",None,None
4,"b'On Wed Aug 21 2002 at 15:46, Ulises Ponce wr...",None,None
...,...,...,...
2546,"b""URL: http://www.newsisfree.com/click/-3,8714...",None,None
2547,"b'URL: http://www.newsisfree.com/click/-4,8723...",None,None
2548,"b'URL: http://www.newsisfree.com/click/-3,8710...",None,None
2549,"b'URL: http://www.newsisfree.com/click/-4,8724...",None,None


In [115]:
hamdf.dropna(axis=1, inplace=True)

In [116]:
hamdf['labels'] = 0

In [117]:
hamdf

,0,labels
0,"b' Date: Wed, 21 Aug 2002 10:54:46 -...",0
1,"b""Martin A posted:\nTassos Papadopoulos, the G...",0
2,"b""Man Threatens Explosion In Moscow \n\nThursd...",0
3,"b""Klez: The Virus That Won't Die\n \nAlready t...",0
4,"b'On Wed Aug 21 2002 at 15:46, Ulises Ponce wr...",0
...,...,...
2546,"b""URL: http://www.newsisfree.com/click/-3,8714...",0
2547,"b'URL: http://www.newsisfree.com/click/-4,8723...",0
2548,"b'URL: http://www.newsisfree.com/click/-3,8710...",0
2549,"b'URL: http://www.newsisfree.com/click/-4,8724...",0


In [124]:
spamdf = pd.DataFrame(spam_mail)

In [126]:
spamdf = spamdf.T

In [127]:
spamdf

,0,1,2
0,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 ...",None,None
1,"b""1) Fight The Risk of Cancer!\nhttp://www.adc...",None,None
2,"b""1) Fight The Risk of Cancer!\nhttp://www.adc...",None,None
3,"b""############################################...",None,None
4,"b""I thought you might like these:\n1) Slim Dow...",None,None
...,...,...,...
495,"b""There is NO stumbling on to it! \n\nThe grea...",None,None
496,"b"" What does it mean to be made in the USA?\n ...",Hit the Road with CNA We encourage you to atte...,None
497,"b'<html>\n\n<head>\n<meta http-equiv=""Content-...",None,None
498,"b'<HTML>\n<BODY>\n <tr valign=""top""> \n <t...",None,None


In [128]:
spamdf.dropna(axis=1, inplace=True)

In [129]:
spamdf['labels'] =1

In [130]:
spamdf

,0,labels
0,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 ...",1
1,"b""1) Fight The Risk of Cancer!\nhttp://www.adc...",1
2,"b""1) Fight The Risk of Cancer!\nhttp://www.adc...",1
3,"b""############################################...",1
4,"b""I thought you might like these:\n1) Slim Dow...",1
...,...,...
495,"b""There is NO stumbling on to it! \n\nThe grea...",1
496,"b"" What does it mean to be made in the USA?\n ...",1
497,"b'<html>\n\n<head>\n<meta http-equiv=""Content-...",1
498,"b'<HTML>\n<BODY>\n <tr valign=""top""> \n <t...",1


In [168]:
df = [hamdf, spamdf]
email = pd.concat(df, ignore_index=True)

In [169]:
email.head()

,0,labels
0,"b' Date: Wed, 21 Aug 2002 10:54:46 -...",0
1,"b""Martin A posted:\nTassos Papadopoulos, the G...",0
2,"b""Man Threatens Explosion In Moscow \n\nThursd...",0
3,"b""Klez: The Virus That Won't Die\n \nAlready t...",0
4,"b'On Wed Aug 21 2002 at 15:46, Ulises Ponce wr...",0


In [170]:
email.rename(columns={0:'Message'}, inplace=True)

In [171]:
email.describe()

,labels
count,3051.000000
mean,0.163881
std,0.370228
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [172]:
email.head()

,Message,labels
0,"b' Date: Wed, 21 Aug 2002 10:54:46 -...",0
1,"b""Martin A posted:\nTassos Papadopoulos, the G...",0
2,"b""Man Threatens Explosion In Moscow \n\nThursd...",0
3,"b""Klez: The Virus That Won't Die\n \nAlready t...",0
4,"b'On Wed Aug 21 2002 at 15:46, Ulises Ponce wr...",0


In [180]:
email['Message'] = email['Message'].astype(str)

In [185]:
email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051 entries, 0 to 3050
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  3051 non-null   object
 1   labels   3051 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.8+ KB


In [173]:
stopword = nltk.corpus.stopwords.words('english')

In [174]:
wn = nltk.WordNetLemmatizer()

In [200]:
def clean_text(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopword]
    return text

In [201]:
email['clean'] = email['Message'].apply(lambda x: clean_text(x))

In [202]:
email

,Message,labels,clean
0,"b' Date: Wed, 21 Aug 2002 10:54:46 -...",0,"[, date, wed, 21, aug, 2002, 10, 54, 46, 0500,..."
1,"b""Martin A posted:\nTassos Papadopoulos, the G...",0,"[, martin, posted, ntassos, papadopoulos, gree..."
2,"b""Man Threatens Explosion In Moscow \n\nThursd...",0,"[, man, threatens, explosion, moscow, nthursda..."
3,"b""Klez: The Virus That Won't Die\n \nAlready t...",0,"[, klez, virus, die, nalready, prolific, virus..."
4,"b'On Wed Aug 21 2002 at 15:46, Ulises Ponce wr...",0,"[, wed, aug, 21, 2002, 15, 46, ulises, ponce, ..."
...,...,...,...
3046,"b""There is NO stumbling on to it! \n\nThe grea...",1,"[, stumbling, nthe, greatest, way, marketing, ..."
3047,"b"" What does it mean to be made in the USA?\n ...",1,"[, mean, made, usa, cna, hitting, road, tell, ..."
3048,"b'<html>\n\n<head>\n<meta http-equiv=""Content-...",1,"[, html, head, meta, http, equiv, content, lan..."
3049,"b'<HTML>\n<BODY>\n <tr valign=""top""> \n <t...",1,"[, html, body, tr, valign, top, td, height, 29..."


In [204]:
mail = email.drop('Message', axis=1)

In [205]:
mail.to_csv('mail.csv')

In [206]:
mail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051 entries, 0 to 3050
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  3051 non-null   int64 
 1   clean   3051 non-null   object
dtypes: int64(1), object(1)
memory usage: 47.8+ KB


In [207]:
mail.head()

,labels,clean
0,0,"[, date, wed, 21, aug, 2002, 10, 54, 46, 0500,..."
1,0,"[, martin, posted, ntassos, papadopoulos, gree..."
2,0,"[, man, threatens, explosion, moscow, nthursda..."
3,0,"[, klez, virus, die, nalready, prolific, virus..."
4,0,"[, wed, aug, 21, 2002, 15, 46, ulises, ponce, ..."
